Note: due to problem with symbolic links lookups and permission(shortcuts) these files are simply removed.

# Define Paths

In [2]:
from pathlib import Path
source_path = Path("X:/Nearline Spruston 2020")
dest_path = Path("Z:/Legacy nearline backup/Nearline Spruston 2020")


In [ ]:
import filecmp
from IPython.display import display, clear_output
import shutil
import time
import glob

def unlink(file_obj,attempts=5):
    success=False
    attempt=0
    while success==False & attempt<attempts:
        try:
            file_obj.unlink()
            success=True
        except:
            attempt+=1
            time.sleep(3)
    if success==False:
        raise NameError(f"Could not unlink {file_obj} after {attempt} attempts")
    #print(f"Unlink {file_obj}: {success} - {attempt}")

def copy(source_obj, destination_obj,attempts=5):
    success=False
    attempt=0
    while success==False & attempt<attempts:
        try:
            shutil.copy2(source_obj, destination_obj)
            success=True
        except:
            attempt+=1
            time.sleep(3)
    if success==False:
        raise NameError(f"Could not copy {source_obj} after {attempt} attempts")
    #print(f"Copy {file_obj}: {success} - {attempt}")

def stat(source_obj, attempts=5):
    success=False
    attempt=0
    stat = None
    while success==False & attempt<attempts:
        try:
            stat = source_obj.stat()
            success=True
        except:
            attempt+=1
            time.sleep(3)
    if success==False:
        raise NameError(f"Could not get stats {file_obj} after {attempt} attempts")
    return stat

def cmp(source_obj, destination_obj, attempts=5):
    success=False
    attempt=0
    while success==False & attempt<attempts:
        try:
            is_equal = filecmp.cmp(source_obj,destination_obj,shallow=True)
            success=True
        except:
            attempt+=1
            time.sleep(3)
    if success==False:
        raise NameError(f"Could not compare {source_obj} after {attempt} attempts")
    else:
        return is_equal

# ensure destination folder exists.
dest_path.mkdir(parents=True, exist_ok=True)
# get all files.
total_moved_gb = 0
total_moved_files = 0
for source_file_str in glob.iglob((source_path/'**').as_posix(), recursive=True):
    start = time.time()
    source_file = Path(source_file_str)
    # check file types
    is_symlink = source_file.is_symlink()
    if is_symlink:
        is_file = True # treat symlink as file, shutil  opy follow_symlinks=False
        is_dir = False
    else:
        is_file = source_file.is_file()
        is_dir = source_file.is_dir()     
        

    # if this is a folder check if its empty.
    if is_dir:
        if not any(source_file.iterdir()):
            source_file.rmdir()
    # check if file.
    if is_file:
        # Get relative file path.
        relative_file_path = source_file.relative_to(source_path)
        destination_file = dest_path/relative_file_path
        # check if file already exists.
        if destination_file.is_file():
            # check if files are the same size.
            if filecmp.cmp(source_file,destination_file,shallow=True):
                # delete source file.
                remove_file = True
                move_file = False
            else:
                remove_file = False
                move_file = True
        # file not yet present
        else:
            remove_file = False
            move_file = True
        # copy file.
        if move_file & (is_symlink==False):
            # check if directory exists.
            destination_file.parent.mkdir(parents=True, exist_ok=True)
            copy(source_file,destination_file)
            # check succesfull copy.
            if cmp(source_file,destination_file):
                remove_file = True
            else:
                remove_file = False
        if remove_file | is_symlink:
            total_moved_files+=1
            if not is_symlink: file_size_gb = stat(source_file).st_size*10**-9
            else: file_size_gb = 0
            total_moved_gb += file_size_gb
            unlink(source_file)
            clear_output(wait=True)
            display(f"Total moved: {total_moved_files} files, {total_moved_gb:.3f} GB")
            display(f"Last file: {relative_file_path} ({file_size_gb:.3f} GB in {time.time()-start:.2f} seconds)")

'Total moved: 97 files, 0.001 GB'

'Last file: from_tier2\\sat\\240624_3\\trakem2.1398465726479.566294465.1138556591\\trakem2.mipmaps\\4\\12\\38\\25.A3_60_2.tif.rag (0.000 GB in 1.57 seconds)'

# Appendix
## Create test data

In [15]:
def create_dummy_file(file_loc,file_size_mb):
    with open(file_loc, 'wb') as f:
        f.seek(file_size_mb * 1024 * 1024) # One GB
        f.write(b'\x21')

### Create 5 dummy files in source

In [100]:
for i in range(5):
    create_dummy_file(source_path/f"{i}.dummy",100)

In [66]:
for i in range(5):
    create_dummy_file(dest_path/f"{i}.dummy",50)